Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

# In this problem we will use family dataset and write some datalog rules.

## Few points:
* Refer [Clingo with Jupyter Intro](Clingo_with_Jupyter_Intro.ipynb) before attempting this notebook.
* It's important to run following cell first for rest of notebook to work.
* It's always a good idea to run cells in order. In case you have run cells in jumbled order and would want to start fresh, restart kernel from menu above.
* All clingo cells start with `%%clingo`.
* You can run your clingo cell against some basic facts and rules from a file. `set_db_file $filepath` sets the file against which your clingo cells will run.
* Each clingo cell is independent of others. Rules defined in one cell won't be available in others.
* It's nice to be able to execute clingo from within your notebook but don't forget to practice from command line. `%%clingo` is just a thin wrapper over command line and it's best to know how to use the underlying tool.
* Upon assignment submission, we will run your code against different set of facts. Please don't hardcode answers and save yourself the embarassment.

### Good luck!!

In [2]:
%reload_ext lib.clingo.clingo_magic
import os
from lib.clingo.clingo_evaluate_util import clingo_evaluate

In [3]:
# All clingo cells will run against this file containing some base facts.
family_base_facts_and_rules_file = os.path.expanduser('~/data_readonly/datalog/family_base.lp')
%set_db_file $family_base_facts_and_rules_file

'% EDB (Extensional Database), "Facts"\n% parent(P, C) <=> P is_parent_of C\nparent(william, john).\nparent(john, james).\nparent(james, bill).\nparent(sue, bill).\nparent(james, carol).\nparent(sue, carol).\n\nmale(john).\nmale(james).\nmale(bill).\n\nfemale(sue).\nfemale(carol).\n\n% IDB (Intensional Database) / RULES\ngrandparent(X,Y) :- parent(X, Z), parent(Z, Y).\nfather(X,Y)  :-     parent(X, Y), male(X).\nmother(X,Y)  :-     parent(X, Y), female(X).\nbrother(X,Y) :-     parent(P, X), parent(P, Y), male(X), X != Y.\nsister(X,Y)  :-     parent(P, X), parent(P, Y), female(X), X != Y.'

## We will now write various rules one by one

### [12.5 points] descendant

In [4]:
%%clingo {"predicate" : "descendant", "predicate_arity" : 2, "result_var": "Descendant"}

% Change following expression.
descendant(X,Y) :- parent(Y,X).
descendant(X,Z) :- parent(Y,X), descendant(Y,Z).



Saving output to local variable Descendant['result']
Saving code snippet to local variable Descendant['code']


'descendant(john,william) descendant(james,john) descendant(bill,james) descendant(bill,sue) descendant(carol,james) descendant(carol,sue) descendant(bill,john) descendant(carol,john) descendant(james,william) descendant(bill,william) descendant(carol,william)'


#### [5 points] Test 1 for descendant.
Following test will compare output of your descendant rule against expected output.
You must have run all clingo cells above for test to pass.

In [5]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''
descendant(john,william) descendant(james,john) descendant(bill,james) descendant(bill,sue) descendant(carol,james) descendant(carol,sue) descendant(carol,john) descendant(bill,john) descendant(james,william) descendant(bill,william) descendant(carol,william)
'''
db_file = os.path.expanduser('~/data_readonly/datalog/family_base.lp')
clingo_evaluate(db_file, Descendant['code'], 'descendant', 2, expected_output)

'descendant(john,william) descendant(james,john) descendant(bill,james) descendant(bill,sue) descendant(carol,james) descendant(carol,sue) descendant(carol,john) descendant(bill,john) descendant(james,william) descendant(bill,william) descendant(carol,william)'


#### [7.5 points] Test 2 for descendant.
Following is what is called a hidden test case. This will always pass in student's version but will actually be evaluated after submission.
* We will first add some facts that are hidden from student.
* We will run descendant rule using these new facts and see if rule still behaving correctly.

In [6]:
# This cell will test the descendant with these new facts.
# Contents of this cell will not be present in student's version of assignment.
# This will only be evaluated after submission.


### [12.5 points] sibling

In [7]:
%%clingo {"predicate" : "sibling", "predicate_arity" : 2, "result_var": "Sibling"}

% Following code snippet and it's result will be assigned to local variable Sibling

% SIBLING
% X and Y are siblings if they share a parent A.
% Change following expression.
sibling(X,Y) :- parent(P, X), parent(P, Y), X < Y.



Saving output to local variable Sibling['result']
Saving code snippet to local variable Sibling['code']


'sibling(bill,carol)'


#### [5 points] Test for sibling.
Following test will compare output of your sibling rule against expected output.
You must have run all clingo cells above for test to pass.

In [8]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''
sibling(bill,carol)
'''
db_file = os.path.expanduser('~/data_readonly/datalog/family_base.lp')
clingo_evaluate(db_file, Sibling['code'], 'sibling', 2, expected_output)

'sibling(bill,carol)'

#### [7.5 points] Test 2 for sibling.
Following is what is called a hidden test case. This will always pass in student's version but will actually be evaluated after submission.
* We will first add some facts that are hidden from student.
* We will run sibling rule using these new facts and see if rule still behaving correctly.

In [9]:
# This cell will test the sibling with these new facts.
# Contents of this cell will not be present in student's version of assignment.
# This will only be evaluated after submission.


### [12.5 points] icv_person_has_parent

In [10]:
%%clingo {"predicate" : "icv_person_has_parent", "predicate_arity" : 1, "result_var": "Icv_person_has_parent"}

% Following code snippet and it's result will be assigned to local variable Icv_person_has_parent

% Change following expression.
% First find all persons; then find who has a parent; then who hasn't (using a safe rule).

% http://www.dlvsystem.com/html/The_DLV_Tutorial.html
person(X) :- parent(X, _).
person(X) :- parent(_, X).
person_has_parent(A) :- parent(_, A).
icv_person_has_parent(A) :- person(A), not person_has_parent(A).



Saving output to local variable Icv_person_has_parent['result']
Saving code snippet to local variable Icv_person_has_parent['code']


'icv_person_has_parent(william) icv_person_has_parent(sue)'


#### [5 points] Test 1 for icv_person_has_parent.
Following test will compare output of your icv_person_has_parent rule against expected output.
You must have run all clingo cells above for test to pass.

In [11]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''
icv_person_has_parent(william) icv_person_has_parent(sue)
'''

db_file = os.path.expanduser('~/data_readonly/datalog/family_base.lp')
clingo_evaluate(db_file, Icv_person_has_parent['code'], 'icv_person_has_parent', 1, expected_output)

'icv_person_has_parent(william) icv_person_has_parent(sue)'


#### [7.5 points] Test 2 for icv_person_has_parent.
Following is what is called a hidden test case. This will always pass in student's version but will actually be evaluated after submission.
* We will first add some facts that are hidden from student.
* We will run icv_person_has_parent rule using these new facts and see if rule still behaving correctly.

In [12]:
# This cell will test the icv_person_has_parent with these new facts.
# Contents of this cell will not be present in student's version of assignment.
# This will only be evaluated after submission.


### [12.5 points] icv_person_has_father_mother

In [13]:
%%clingo {"predicate" : "icv_person_has_father_mother", "predicate_arity" : 1, "result_var": "Icv_person_has_father_mother"}

% Following code snippet and it's result will be assigned to local variable Icv_person_has_father_mother

% Change following expression.
% Find all people having both a father and mother, then report those who haven't.
person(X) :- parent(X, _).
person(X) :- parent(_, X).
person_has_father(A) :- father(_, A).
person_has_mother(A) :- mother(_, A).
icv_person_has_father_mother(A) :- person(A), not person_has_father(A).
icv_person_has_father_mother(A) :- person(A), not person_has_mother(A).




Saving output to local variable Icv_person_has_father_mother['result']
Saving code snippet to local variable Icv_person_has_father_mother['code']


'icv_person_has_father_mother(william) icv_person_has_father_mother(john) icv_person_has_father_mother(james) icv_person_has_father_mother(sue)'

#### [5 points] Test 1 for icv_person_has_father_mother.
Following test will compare output of your icv_person_has_father_mother rule against expected output.
You must have run all clingo cells above for test to pass.

In [14]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''
icv_person_has_father_mother(william) icv_person_has_father_mother(john) icv_person_has_father_mother(james) icv_person_has_father_mother(sue)


'''

db_file = os.path.expanduser('~/data_readonly/datalog/family_base.lp')
clingo_evaluate(db_file, Icv_person_has_father_mother['code'], 'icv_person_has_father_mother', 1, expected_output)

'icv_person_has_father_mother(william) icv_person_has_father_mother(john) icv_person_has_father_mother(james) icv_person_has_father_mother(sue)'


#### [7.5 points] Test 2 for icv_person_has_father_mother.
Following is what is called a hidden test case. This will always pass in student's version but will actually be evaluated after submission.
* We will first add some facts that are hidden from student.
* We will run icv_person_has_father_mother rule using these new facts and see if rule still behaving correctly.

In [15]:
# This cell will test the icv_person_has_father_mother with these new facts.
# Contents of this cell will not be present in student's version of assignment.
# This will only be evaluated after submission.
